<a href="https://colab.research.google.com/github/NataIss/codeforces/blob/main/Codeforces_15_NumOfUsersByContest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import pandas as pd
import numpy as np
import time

In [ ]:
try:  
    data = pd.read_json("https://codeforces.com/api/contest.list?gym=false")
    con = pd.DataFrame.from_records(data['result'])
    con
except (BaseException) as e:    
    print(e)
con

In [ ]:
con=con[(con['type']=='CF') & (con['phase']=='FINISHED')]
con

In [ ]:
 link='https://codeforces.com/api/contest.ratingChanges?contestId=1625'
 data = pd.read_json(link)
 if data['status'][0]=='OK':
       dfAll = pd.DataFrame.from_records(data['result'])

In [ ]:
for j in con['id']: 
    i=0
    while i<3:
        try:
            time.sleep(0.2)
            link='https://codeforces.com/api/contest.ratingChanges?contestId=' + str(j)
            data = pd.read_json(link)
            if data['status'][0]=='OK':
                df = pd.DataFrame.from_records(data['result'])
                dfAll=pd.concat([dfAll,df],ignore_index=True)
                i=i+5
                print(j) + "****"
        except (BaseException) as e:    
                time.sleep(1)
                i=i+1
                #ratedListAll[ratedListAll['handle']==j][0]
                if i==3:
                    print(j)
                    print(e)
dfAll

In [ ]:
dfAll.drop_duplicates(keep='last',subset=['handle','contestId'],inplace=True)

In [ ]:
w=pd.DataFrame(dfAll.groupby('contestId')['rank'].count())
w.head()

In [ ]:
con.set_index('id',inplace=True)

In [ ]:
res = pd.merge(con, w, how="outer",left_index=True,right_index=True)
res

In [ ]:
res=res.fillna(0)
res=res[res['rank']>0]
res.rename(columns={'rank':'NumOfUsers'}, inplace=True)

In [ ]:
res1=res[['name','startTimeSeconds','NumOfUsers']]
res1.head()

In [ ]:
res1.to_excel('NumOfUsersByContest.xlsx')